# Ettoday 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例：

1. 取出今天所有的新聞
2. 取出現在時間兩小時內的新聞
3. 根據範例，取出三天前下午三點到五點的新聞

In [66]:
import requests
import time
import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select

In [2]:
Today = datetime.datetime.now().strftime('%Y/%m/%d')
Today                                        

'2020/02/07'

In [30]:
browser = webdriver.Chrome(executable_path='chromedriver')

browser.get('https://www.ettoday.net/news/news-list.htm')

## 1. 取出今天所有的新聞

In [71]:
#思路: 發現某些新聞的時間有誤，因此沒辦法用時間來制定終止點，故先把所有可以挖的新聞都先挖出後再進行過濾
def autoscroll(t = 2):
    last_news = ''
    while True:
        time.sleep(t) #由於某些新聞挖的很慢，因此設定長一點(為了等待網頁載入完成!! 否則會出錯)
        browser.execute_script("window.scrollTo(0, 100000);")
        html_source = browser.page_source
        soup = BeautifulSoup(html_source, 'lxml')
        NewsList = soup.find(class_ = 'part_list_2').find_all('a')
        if last_news == NewsList[-1]:
            break
        last_news = NewsList[-1]
    return soup

In [51]:
soup = autoscroll(t = 10)
Today_news = dict()
for tmp in soup.find_all('h3'):
    time = tmp.find(class_ = 'date')
    if time and time.text[:10] == Today:
        Today_news[tmp.find('a').text] = time.text
print("今天日期為: {},以下列出今天的新聞\n".format(Today))
for news, time in Today_news.items():
    print(news, time)

今天日期為: 2020/02/07,以下列出今天的新聞

北部疫情地點曝！1小時嚇慘280萬人　一招教你「google地圖」查詢 2020/02/07 21:16
首個實施「封閉式管理」直轄市　天津下令社區關閉非必要進出通道 2020/02/07 21:12
武漢肺炎疫情民眾莫驚慌　網傳防疫妙招多是不正確迷思要注意 2020/02/07 21:11
最後一天！春節大紅包「7700萬」一次發完　大樂透3.6億號碼開出 2020/02/07 21:11
紐約直播／rag ＆ bone思考地球怎麼了　2／8早上8點看秋冬秀 2020/02/07 21:08
立院新會期最快21日開議　將處理2大預算、3項人事同意權案 2020/02/07 21:07
Ella「1個約定記4年」說到做到　電話另一頭...阿Ken哭了 2020/02/07 21:07
日牌受武漢肺炎打擊　UNIQLO、無印良品大陸分店已過半數暫停營業 2020/02/07 20:59
Sunny、孝敏曬合照「一直陪在身邊的10年好友」　勾G7忠實粉回憶：《青春不敗》萬歲！ 2020/02/07 20:58
單身才是王道！　12星座拒絕戀愛原因…天秤摩羯「還想玩、不願負責」 2020/02/07 20:51
富士康致信深圳廠員工10日暫不回廠　鴻海聲明：配合做好安全復工準備 2020/02/07 20:50
日綜驚傳靈異放送事故！罹癌新娘放婚禮影片…冒出男人氣音「去死」網一聽抖了 2020/02/07 20:48
網傳李文亮妻子求助書！本人出面闢謠「不接受任何人捐款」稱不實消息 2020/02/07 20:44
梅西和巴塞隆納緣將盡？　曼城被認最有望下家 2020/02/07 20:43
養父母承受不斷離別！導盲犬只能養到1歲　審核未過也禁回寄養家庭 2020/02/07 20:36
暑假延後放！星宇航空公布改、退票辦法　限定期間內免收手續費 2020/02/07 20:30
影／台股該危機入市？　分析師建議可留意低本益比受惠股 2020/02/07 20:28
「國家級警報」揭染疫遊客足跡！遍及北北基50點　重要景點一覽 2020/02/07 20:27
2對情侶摩鐵開趴「狂嗨21小時」！20歲女疑吸毒助興暴斃死在床上 2020/02/07 20:21
2國限制日本入境！　菅義偉：尊重這幾國「極易感染」立場 

## 2. 取出現在時間兩小時內的新聞

In [62]:
Today_Full = datetime.datetime.now()
for news, time in Today_news.items():
    if (Today_Full - datetime.datetime.strptime(time, "%Y/%m/%d %H:%M")) < datetime.timedelta(hours=2):
        print(news)

北部疫情地點曝！1小時嚇慘280萬人　一招教你「google地圖」查詢
首個實施「封閉式管理」直轄市　天津下令社區關閉非必要進出通道
武漢肺炎疫情民眾莫驚慌　網傳防疫妙招多是不正確迷思要注意
最後一天！春節大紅包「7700萬」一次發完　大樂透3.6億號碼開出
紐約直播／rag ＆ bone思考地球怎麼了　2／8早上8點看秋冬秀
立院新會期最快21日開議　將處理2大預算、3項人事同意權案
Ella「1個約定記4年」說到做到　電話另一頭...阿Ken哭了


## 3. 根據範例，取出三天前下午三點到五點的新聞

In [90]:
#此處暫時不考慮跨月份
selectday = Select(browser.find_element_by_id("selD"))
electday.select_by_value(str(Today_Full.day - 3)) #三天前
browser.find_element_by_id('button').click()

soup = autoscroll()
#不優之方法: 利用第一筆資料來獲得欲查詢之日期
getday = soup.find(class_ = 'date').text[:10]
time15 = datetime.datetime.strptime("15:00", "%H:%M") #下午三點的datetime
time17 = datetime.datetime.strptime("17:00", "%H:%M") #下午五點的datetime

print("{}下午3點到5點之新聞如下:\n".format(getday))
for tmp in soup.find_all('h3'):
    time = tmp.find(class_ = 'date')
    if time and time.text[:10] == getday:
        newstime = datetime.datetime.strptime(time.text[11:], "%H:%M")
        if time15 <= newstime and newstime <= time17:
            #print(time.text) #debug查看時間用
            print(tmp.find('a').text)

2020/02/04下午3點到5點之新聞如下:

太罪惡！首爾隱藏版餐酒館　「奶油辣醬牛小排義大利麵」超過癮
WHO估潛伏期<10天　日本考慮放寬入境標準、縮短隔離期
三星Galaxy S20系列定價曝光！　S20 Ultra最高突破4萬元
功夫汪學爬樹！「海豚式騰飛」帥奪黃球　網驚：少林寺養的？
新光慶元宵做公益　晚會邀方泂鑌壓軸熱唱
aifian攜手嘟嘟房以「數據經紀人AI嘉嘉」　實現汽機車「秒貸」Fintech服務
命理師曝疫情趨緩時間！今「立春」洩天機　宜甜少酸增陽氣
宅在家動手指防疫好安心！台拍網站逛好逛滿　2020經典藍色系穿搭時尚上身
武漢肺炎免驚！房仲分析看屋量逆勢上升　「市場還算理性」
6搶1／中國想來台參賽　致電棒協關心武漢肺炎防疫措施
快問快答／柯文哲爆武漢台商隔離地　63.6％網友：支持公開其他4處
捷運站出口整修將影響人潮　金鳳錦燒鵝開業半年就結束營業
貨運男勾上人妻…討裸照要第一次！對話「腦公♥腦婆」網也惱：腦什麼腦
放棄高薪職位勇敢追夢！Julia談創業：如果可以不會踏上這條路
她棄繁星推甄國立護校！進環球科大勇奪資通電軍女狀元...畢業起薪49K
水電工夫「10天洗一次澡還不刷牙」　嫩妻臭到離婚：他毀了我人生
徵召未有消息　6搶1有約旅外球員參賽有難度　
謠言抹黑民進黨發國難財　顏若芳提告加重誹謗：盼各界自重
亞裔女戴口罩坐地鐵樓梯間　被用雨傘狂毆...網：在歐美戴口罩有病
影／交警攔下怪異司機…他惜命「戴十多層口罩」開車！網狂轟：沒常識又浪費
與愛妻分居「回家4天就吵架」　他提愛人頭顱逛大街…邊唱歌邊揮手嚇呆路人
生力軍來了！16名國軍支援華新口罩廠　日增3萬片產能
獨／送萬元名錶不放在眼裡　得標網友更在乎連千毅前老闆「薄薄一片」　
越南武漢肺炎患者出院　治療方法「非WHO推薦」漱口成關鍵！
世界最棒的衝浪聖地！南九州4大景點　必訪藏在深山的溫泉秘境
防疫打亂航班　華航官網建服務專區「航班、票務一手掌握！」
范瑋琪「口罩事件」慘遭出征圍剿　歌手好友出聲痛斥：拿出良心做人吧！
台南最凶「入住索命」宅新屋主嚇壞急脫手　開價竟想賺一筆
夢醒時分！「花王」遭誤認防疫股暴漲3天...今殺跌停
閨蜜坐男友床上「抱狗示威」　正宮女星Dora被嗆：妳不用在意
台中釣客違規釣魚遭制止　怒嗆海巡「強龍不壓地頭蛇，要練身體嗎？」
滿屋子黃金！老媽意外發現